In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
import re
import requests
import json
import pandas as pd
import ast

In [81]:
# Access spotify api
client_id = '96dcd39191cf46ee81bb5d1092e6716b'
client_secret = '22a0463e07e2485c8fc0a692d3412183'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [82]:
# access genius api
genius_api_key = 'oDCXKglQ5KEKSWTKa3qZBceVqITk6079PgdZI0l0SwJ_UohJzomNzTC_kpQsSs4_'
genius = lyricsgenius.Genius(genius_api_key)

# Popularity Index

The popularity score in the Spotify API is a number between 0 and 100 that tells you how popular a track is on Spotify. A higher score means the track is more popular, and the score is based on factors like the number of plays and the number of times the track has been added to playlists.

In [ ]:
# Search for a track and get its ID
track_name = 'Thriller' # Thriller by Michael Jackson 
results = sp.search(q=track_name, type='track')
track_id = results['tracks']['items'][0]['id']

# Get the track's information and extract its number of streams
track_info = sp.track(track_id)
popularity_score = track_info['popularity']
print(f"{track_name} has a popularity score of {popularity_score}.")

In [ ]:
# Search for a track and get its ID
track_name = 'Creepin\'' # Creepin' by Metro Boomin
results = sp.search(q=track_name, type='track')
track_id = results['tracks']['items'][0]['id']

# Get the track's information and extract its number of streams
track_info = sp.track(track_id)
popularity_score = track_info['popularity']
print(f"{track_name} has a popularity score of {popularity_score}.")

# Artist 

In [ ]:
# get the track name and artist name
track_name = results['tracks']['items'][0]['name']
artist_name = results['tracks']['items'][0]['artists'][0]['name']

In [ ]:
print(artist_name)

# Genre

In [ ]:
# Set up the Last.fm API endpoint and parameters
api_key = '99ada5f533810124997231207a4df132' # Replace with your Last.fm API key
base_url = 'http://ws.audioscrobbler.com/2.0/'
method = 'track.getInfo'
artist = 'Queen' # Replace with the name of the artist
track = 'Bohemian Rhapsody' # Replace with the title of the song you want to search for
params = {
    'method': method,
    'api_key': api_key,
    'artist': artist,
    'track': track,
    'format': 'json'
}

# Send the API request and get the response
response = requests.get(base_url, params=params)
data = json.loads(response.text)

# Extract the genres from the response
tags = data['track']['toptags']['tag']
genres = [tag['name'] for tag in tags]

# Print the genres
print(f"The genres of the song are: {genres}")

# Primary Genres

In [2]:
df = pd.read_excel("Dataframe with Genre.xlsx")

In [3]:
df.shape

(2300, 7)

In [4]:
df.head()

,Unnamed: 0,Year,Rank,Artist,Song,Song_genre,Genre
0,0,2000,1,Faith Hill,Breathe,Breathe,"['country', 'female vocalists', 'pop', 'Love',..."
1,1,2000,2,Santana,Smooth,Smooth,"['rock', 'latin', 'Latin Rock', 'guitar', 'cla..."
2,2,2000,3,Santana,Maria Maria,Maria Maria,"['latin', 'rock', 'santana', 'guitar', 'Latin ..."
3,3,2000,4,Joe,I Wanna Know,I Wanna Know,"['rnb', 'joe', 'slow jams', 'soul', 'smooth']"
4,4,2000,5,Vertical Horizon,Everything You Want,Everything You Want,"['alternative rock', 'rock', '90s', 'alternati..."


In [5]:
# remove index column
del df["Unnamed: 0"]

In [6]:
# use ast.literal_eval to safely convert the strings to lists
df['Genre'] = df['Genre'].apply(lambda x: ast.literal_eval(x) if x != 'NO GENRE' else [])

main_genres = ["pop", "rap", "country", "latin", "rock", "rnb", "r&b", "hip hop", "gospel", "rumba"
              "alternative", "indie", "soul", "reggae", "jazz", "Hip-Hop", "electric", "christmas",]

# filter the dataframe to only include rows where the genres list contains any genre in the main genre
df_filtered = df[df['Genre'].apply(lambda x: any(g in x for g in main_genres))]

# define a function to extract the relevant genres from the list
def extract_genres(genre_list):
    return [g for g in genre_list if g in main_genres]

# apply the function to the filtered dataframe to extract the relevant genres
df_filtered['relevant_genres'] = df_filtered['Genre'].apply(extract_genres)

C:\Users\nbrit\AppData\Local\Temp\ipykernel_85776\2519109591.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['relevant_genres'] = df_filtered['Genre'].apply(extract_genres)


In [7]:
# Look at new df with main genres
df_filtered

,Year,Rank,Artist,Song,Song_genre,Genre,relevant_genres
0,2000,1,Faith Hill,Breathe,Breathe,"[country, female vocalists, pop, Love, Faith H...","[country, pop]"
1,2000,2,Santana,Smooth,Smooth,"[rock, latin, Latin Rock, guitar, classic rock]","[rock, latin]"
2,2000,3,Santana,Maria Maria,Maria Maria,"[latin, rock, santana, guitar, Latin Rock]","[latin, rock]"
3,2000,4,Joe,I Wanna Know,I Wanna Know,"[rnb, joe, slow jams, soul, smooth]","[rnb, soul]"
4,2000,5,Vertical Horizon,Everything You Want,Everything You Want,"[alternative rock, rock, 90s, alternative, pop]","[rock, pop]"
...,...,...,...,...,...,...,...
2287,2022,88,Jason Aldean,Trouble with a Heartbreak,Trouble with a Heartbreak,"[country, Ballad, 2020s, 2022]",[country]
2288,2022,89,Burl Ives,A Holly Jolly Christmas,A Holly Jolly Christmas,"[christmas, christmas classics, xmas, holiday,...",[christmas]
2289,2022,90,Doja Cat,Kiss Me More,Kiss Me More,"[pop, dance, rnb, Disco, electronic]","[pop, rnb]"
2292,2022,93,Scotty McCreery,Damn Strait,Damn Strait,"[country, Ballad, 2020s, 2021]",[country]


In [8]:
# Pick the first genre from "relevant genres"
df_filtered["primary_genre"] = df_filtered["relevant_genres"].apply(lambda x: x[0])

C:\Users\nbrit\AppData\Local\Temp\ipykernel_85776\1103570119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["primary_genre"] = df_filtered["relevant_genres"].apply(lambda x: x[0])


In [9]:
# New df with primary_genre
df_filtered

,Year,Rank,Artist,Song,Song_genre,Genre,relevant_genres,primary_genre
0,2000,1,Faith Hill,Breathe,Breathe,"[country, female vocalists, pop, Love, Faith H...","[country, pop]",country
1,2000,2,Santana,Smooth,Smooth,"[rock, latin, Latin Rock, guitar, classic rock]","[rock, latin]",rock
2,2000,3,Santana,Maria Maria,Maria Maria,"[latin, rock, santana, guitar, Latin Rock]","[latin, rock]",latin
3,2000,4,Joe,I Wanna Know,I Wanna Know,"[rnb, joe, slow jams, soul, smooth]","[rnb, soul]",rnb
4,2000,5,Vertical Horizon,Everything You Want,Everything You Want,"[alternative rock, rock, 90s, alternative, pop]","[rock, pop]",rock
...,...,...,...,...,...,...,...,...
2287,2022,88,Jason Aldean,Trouble with a Heartbreak,Trouble with a Heartbreak,"[country, Ballad, 2020s, 2022]",[country],country
2288,2022,89,Burl Ives,A Holly Jolly Christmas,A Holly Jolly Christmas,"[christmas, christmas classics, xmas, holiday,...",[christmas],christmas
2289,2022,90,Doja Cat,Kiss Me More,Kiss Me More,"[pop, dance, rnb, Disco, electronic]","[pop, rnb]",pop
2292,2022,93,Scotty McCreery,Damn Strait,Damn Strait,"[country, Ballad, 2020s, 2021]",[country],country


In [16]:
# Fix hip-hop and rnb naming 
df_filtered.loc[:, "primary_genre"] = df_filtered.apply(lambda row: row.primary_genre if row.primary_genre != "Hip-Hop" else "hip hop", axis=1)
df_filtered.loc[:, "primary_genre"] = df_filtered.apply(lambda row: row.primary_genre if row.primary_genre != "rnb" else "r&b", axis=1)

C:\Users\nbrit\AppData\Local\Temp\ipykernel_85776\2191303749.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, "primary_genre"] = df_filtered.apply(lambda row: row.primary_genre if row.primary_genre != "Hip-Hop" else "hip hop", axis=1)
C:\Users\nbrit\AppData\Local\Temp\ipykernel_85776\2191303749.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, "primary_genre"] = df_filtered.apply(lambda row: row.primary_genre if row.primary_genre != "rnb" else "r&b", axis=1)


In [20]:
df_filtered.head(10)

,Year,Rank,Artist,Song,Song_genre,Genre,relevant_genres,primary_genre
0,2000,1,Faith Hill,Breathe,Breathe,"[country, female vocalists, pop, Love, Faith H...","[country, pop]",country
1,2000,2,Santana,Smooth,Smooth,"[rock, latin, Latin Rock, guitar, classic rock]","[rock, latin]",rock
2,2000,3,Santana,Maria Maria,Maria Maria,"[latin, rock, santana, guitar, Latin Rock]","[latin, rock]",latin
3,2000,4,Joe,I Wanna Know,I Wanna Know,"[rnb, joe, slow jams, soul, smooth]","[rnb, soul]",r&b
4,2000,5,Vertical Horizon,Everything You Want,Everything You Want,"[alternative rock, rock, 90s, alternative, pop]","[rock, pop]",rock
5,2000,6,Destiny’s Child,Say My Name,Say My Name,"[rnb, pop, destinys child, female vocalists, 90s]","[rnb, pop]",r&b
6,2000,7,Savage Garden,I Knew I Loved You,I Knew I Loved You,"[pop, 90s, savage garden, Love, love songs]",[pop],pop
7,2000,8,Lonestar,Amazed,Amazed,"[country, lonestar, love songs, Amazed, Love]",[country],country
8,2000,9,matchbox twenty,Bent,Bent,"[rock, alternative rock, alternative, 90s, pop]","[rock, pop]",rock
9,2000,10,Toni Braxton,He Wasn’t Man Enough,He Wasnt Man Enough,"[female, pop, dance, RB, rnb]","[pop, rnb]",pop


In [17]:
# Count by primary genre
df_filtered.groupby("primary_genre")["Rank"].count().reset_index()

,primary_genre,Rank
0,christmas,7
1,country,161
2,gospel,1
3,hip hop,324
4,indie,43
5,jazz,1
6,latin,13
7,pop,658
8,r&b,241
9,rap,154


In [21]:
# Export the DataFrame to a CSV file
df_filtered.to_csv('genres.csv', index=False)